this notebook is to plot the extended logit lens for the DeepSeek MoE model.

The extended Logit Lens can be defined as:

$$\text{LogitLens}^{\text{ext}}(h_t^\ell) = \text{LayerNorm}(h_t^\ell + u_t^\ell)W_U$$

Where:
- $h_t^\ell$ represents the hidden state of the $t$-th token after the $\ell$-th layer
- $u_t^\ell$ represents the post-attention residual stream for the $t$-th token at layer $\ell$
- $W_U$ is the model's pretrained unembedding matrix
- $\text{LayerNorm}$ is the layer normalization function

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import os
from typing import Dict, List, Tuple
from collections import defaultdict

In [2]:
def load_model(model_name):
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        trust_remote_code=True,
        # use_flash_attention_2=True,
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model.eval()
    return model, tokenizer

model, tokenizer = load_model("deepseek-ai/deepseek-moe-16b-base")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
def get_top_k_tokens(hidden_states: torch.Tensor, lm_head: torch.nn.Linear, tokenizer, k: int = 5) -> List[Tuple[str, float]]:
    """ get topk tokens from hidden states using lm head """
    with torch.no_grad():
        # Ensure hidden_states has at least 2 dimensions (batch_size, num_tokens, hidden_dim)
        if hidden_states.dim() == 2:
            hidden_states = hidden_states.unsqueeze(0)  # Add batch dimension
            # Apply RMS normalization like in DeepseekRMSNorm
            variance = hidden_states.pow(2).mean(-1, keepdim=True) 
            hidden_states = hidden_states * torch.rsqrt(variance + 1e-6)

        logits = lm_head(hidden_states)  # (batch_size, num_tokens, vocab_size)
    
    # Get top-k tokens
    scores, token_ids = torch.topk(logits, k=k, dim=-1)  # (batch_size, num_tokens, k)
    
    # Decode tokens and collect results for each position
    results = []
    for pos in range(scores.size(1)):  # Iterate over token positions
        pos_results = []
        for i in range(k):
            token = tokenizer.decode(token_ids[0, pos, i])  # Decode token for this position
            score = scores[0, pos, i].item()  # Get score for this position
            pos_results.append((token, score))
        results.append(pos_results)
    
    return results

In [4]:
class DeepseekLayerAnalyzer:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.layer_outputs = defaultdict(list)
        self.moe_gate_outputs = defaultdict(list)
        self.moe_combined_outputs = defaultdict(list)
        self.expert_outputs = defaultdict(lambda: defaultdict(list))
        self.shared_expert_outputs = defaultdict(list)
        self.hooks = []
        
    def register_hooks(self):
        """Register hooks for layer outputs and MoE combination points"""
        
        def layer_output_hook(layer_idx):
            def hook(module, inputs, outputs):
                """Hook for capturing layer outputs"""
                hidden_states = outputs[0] if isinstance(outputs, tuple) else outputs
                self.layer_outputs[layer_idx].append(hidden_states.detach())
            return hook

        def moe_gate_hook(layer_idx):
            def hook(module, inputs, outputs):
                """Hook for capturing MoE gate outputs before expert computation"""
                # Capture topk_idx, topk_weight, and aux_loss from gate outputs
                if isinstance(outputs, tuple):
                    topk_idx, topk_weight, aux_loss = outputs
                    self.moe_gate_outputs[layer_idx].append({
                        'topk_idx': topk_idx.detach(),
                        'topk_weight': topk_weight.detach(),
                        'aux_loss': aux_loss.detach() if aux_loss is not None else None
                    })
            return hook

        def expert_hook(layer_idx, expert_idx):
            def hook(module, inputs, outputs):
                """Hook for capturing expert outputs"""
                # Get the latest gate outputs for this layer
                if not self.moe_gate_outputs[layer_idx]:
                    return print(f'no gate outputs for layer {layer_idx}')
            
                gate_data = self.moe_gate_outputs[layer_idx][-1]
                
                # Handle 2D or 3D tensor shapes
                if len(gate_data['topk_idx'].shape) == 2:
                    batch_size = 1
                    seq_len, top_k = gate_data['topk_idx'].shape
                else:
                    batch_size, seq_len, top_k = gate_data['topk_idx'].shape
                
                # Get mask for tokens where this expert was selected
                expert_mask = (gate_data['topk_idx'] == expert_idx)                
                # Flatten and find positions where this expert was selected
                selected_positions = torch.nonzero(expert_mask, as_tuple=True)
                # If no tokens selected this expert, skip
                if selected_positions[0].numel() == 0:
                    return
                    
                # Get the actual inputs routed to this expert
                # Inputs[0] shape: (total_selected_tokens, hidden_dim)
                total_selected = inputs[0].shape[0] 
                # Validate we're processing the correct number of tokens
                expected_selected = expert_mask.sum().item()
                if total_selected != expected_selected:
                    print(f" expert {expert_idx} processed {total_selected} tokens but expected {expected_selected}")
                    return
                    
                # Get the full hidden states from outputs
                # outputs shape: (total_selected_tokens, hidden_dim)
                hidden_states = outputs
                if isinstance(outputs, tuple):
                    hidden_states = outputs[0]
                    
                # Record data for each selected position
                for pos_idx, pos in enumerate(selected_positions[0]):
                    token_data = {
                        'position': pos.item(),
                        'input': inputs[0][pos_idx].detach(),
                        'output': outputs[pos_idx].detach(),
                        'hidden_state': hidden_states[pos_idx].detach()  # Store full hidden state
                    }
                    
                    # Get the corresponding gate weight for this position
                    # Find which expert slot (in top_k) this expert was selected for this position
                    expert_slots = (gate_data['topk_idx'][pos.item()] == expert_idx).nonzero(as_tuple=True)[0]
                    if len(expert_slots) > 0:
                        token_data['gate_weight'] = gate_data['topk_weight'][pos.item()][expert_slots[0]].item()
                    
                    self.expert_outputs[layer_idx][expert_idx].append(token_data)
            return hook
        
        def shared_expert_hook(layer_idx):
            def hook(module, inputs, outputs):
                """Hook for capturing shared expert outputs"""
                self.shared_expert_outputs[layer_idx].append({
                    'input': inputs[0].detach(),
                    'output': outputs.detach()
                })
            return hook

        def moe_combine_hook(layer_idx):
            def hook(module, inputs, outputs):
                """Hook for capturing final combined MoE outputs"""
                # For DeepseekMoE, this captures the weighted sum of expert outputs
                self.moe_combined_outputs[layer_idx].append({
                    'combined_output': outputs.detach(),
                    'input': inputs[0].detach()  # Original input before MoE
                })
            return hook

        # Register hooks for each layer
        for layer_idx, layer in enumerate(self.model.model.layers):
            # Hook for layer output
            hook = layer.register_forward_hook(layer_output_hook(layer_idx))
            self.hooks.append(hook)
            
            # If it's an MoE layer, add MoE-specific hooks
            if hasattr(layer.mlp, 'experts'):
                # Hook for gate mechanism
                gate_hook = layer.mlp.gate.register_forward_hook(moe_gate_hook(layer_idx))
                self.hooks.append(gate_hook)
                
                # Hook for each expert
                for expert_idx, expert in enumerate(layer.mlp.experts):
                    expert_hook_fn = expert.register_forward_hook(expert_hook(layer_idx, expert_idx))
                    self.hooks.append(expert_hook_fn)

                # Hook for shared expert if it exists
                if hasattr(layer.mlp, 'shared_experts'):
                    shared_hook = layer.mlp.shared_experts.register_forward_hook(shared_expert_hook(layer_idx))
                    self.hooks.append(shared_hook)
                
                # Hook for final combined output
                combine_hook = layer.mlp.register_forward_hook(moe_combine_hook(layer_idx))
                self.hooks.append(combine_hook)

    def analyze_tokens(self, input_ids: torch.Tensor, return_hidden_states: bool = False) -> Dict:
        """ run inference and analyze tokens at each layer and expert combination point """

        self.moe_combined_outputs.clear()
        self.expert_outputs.clear()
        self.shared_expert_outputs.clear()
        
        # Forward pass
        with torch.no_grad():
            outputs = self.model(input_ids)
        
        results = {
            'layer_predictions': {},
            'moe_analysis': {},
            'hidden_states': {} if return_hidden_states else None
        }
        
        # Analyze layer outputs
        for layer_idx, outputs in self.layer_outputs.items():
            if not outputs:  # Skip if no outputs captured
                continue
            hidden_states = outputs[-1]  # Get last captured output
            
            
            
            
            # Get token predictions for this layer
            top_tokens = get_top_k_tokens(hidden_states, self.model.lm_head, self.tokenizer)
            results['layer_predictions'][layer_idx] = top_tokens
            
            if return_hidden_states:
                results['hidden_states'][f'layer_{layer_idx}'] = hidden_states
        
        # Analyze MoE layers
        for layer_idx in self.moe_gate_outputs.keys():
            if not self.moe_gate_outputs[layer_idx]:
                continue
                
            gate_data = self.moe_gate_outputs[layer_idx][-1]  # Get last captured data
            combined_data = self.moe_combined_outputs[layer_idx][-1]
            
            # Initialize predictions dictionary by position
            expert_predictions_by_pos = defaultdict(dict)
            expert_hidden_states_by_pos = defaultdict(dict)
            
            # Process expert outputs by position
            for expert_idx, data_list in self.expert_outputs[layer_idx].items():
                for data in data_list:
                    position = data['position']
                    # Get predictions for this expert's output at this position
                    predictions = get_top_k_tokens(
                        data['output'].unsqueeze(0),
                        self.model.lm_head,
                        self.tokenizer
                    )
                    expert_predictions_by_pos[position][expert_idx] = predictions[0]  # [0] because we only have one prediction set

                    # Store hidden states
                    expert_hidden_states_by_pos[position][expert_idx] = {
                    'hidden_state': data['hidden_state'].tolist(),
                    'gate_weight': data.get('gate_weight', None)
                }
            
            # Get predictions for shared expert if it exists
            if self.shared_expert_outputs[layer_idx]:
                shared_expert_predictions = get_top_k_tokens(
                    self.shared_expert_outputs[layer_idx][-1]['output'],
                    self.model.lm_head,
                    self.tokenizer
                )
            
            # Update experts_analysis dictionary to include hidden states
            experts_analysis = {
            'selected_experts': gate_data['topk_idx'].tolist(),
            'expert_weights': gate_data['topk_weight'].tolist(),
            'aux_loss': gate_data['aux_loss'].item() if gate_data['aux_loss'] is not None else None,
            'expert_predictions_by_position': dict(expert_predictions_by_pos),
            'expert_hidden_states_by_position': dict(expert_hidden_states_by_pos),
            'shared_expert_predictions': shared_expert_predictions if self.shared_expert_outputs[layer_idx] else None
        }
            
            # Get token predictions from combined output
            combined_tokens = get_top_k_tokens(
                combined_data['combined_output'], 
                self.model.lm_head,
                self.tokenizer
            )
            
            experts_analysis['combined_output_tokens'] = combined_tokens
            results['moe_analysis'][layer_idx] = experts_analysis

        return results
    
    def cleanup(self):
        """remove all registered hooks"""
        for hook in self.hooks:
            hook.remove()
        self.hooks.clear()

def analyze_deepseek_moe(model, tokenizer, input_text: str, return_hidden_states: bool = False):
    """ analyze DeepSeek MoE model behavior for given input text """
    analyzer = DeepseekLayerAnalyzer(model, tokenizer)
    analyzer.register_hooks()
    
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    
    try:
        results = analyzer.analyze_tokens(input_ids, return_hidden_states=return_hidden_states)
        return results
    finally:
        analyzer.cleanup()  

In [5]:
def get_post_attn_ln_inputs(model, tokenizer, text):
    """places a hook on the post attention layernorm to retrieve its inputs"""
    # Store inputs in a dict mapping layer idx -> inputs
    post_attn_ln_inputs = {}
    hooks = []
    
    def hook_post_attn_ln(module, input, output, layer_idx):
        if layer_idx not in post_attn_ln_inputs:
            post_attn_ln_inputs[layer_idx] = []
        post_attn_ln_inputs[layer_idx].append([x.detach() for x in input])
    
    # Register hooks on post attention layernorm for each layer
    for i, layer in enumerate(model.model.layers):
        hooks.append(
            layer.post_attention_layernorm.register_forward_hook(
                lambda m, i, o, idx=i: hook_post_attn_ln(m, i, o, idx)
            )
        )
    
    try:
        # Run inference
        input_ids = tokenizer(text, return_tensors="pt").input_ids
        model(input_ids)
        
        return post_attn_ln_inputs
        
    finally:
        # Clean up hooks
        for hook in hooks:
            hook.remove()

In [6]:
def get_layer_predictions_for_token(results: dict, layer_idx: int, token_pos: int) -> list:
    # Ensure the layer exists in the results
    if layer_idx not in results['layer_predictions']:
        raise ValueError(f"Layer {layer_idx} not found in the results.")

    # Get the layer predictions for the specified token position
    layer_predictions = results['layer_predictions'][layer_idx][token_pos]
    
    # Return top 5 predictions
    return layer_predictions[:5]

def get_shared_expert_predictions_for_token(results: dict, layer_idx: int, token_pos: int) -> list:
    # Ensure the layer exists in the results
    if layer_idx not in results['moe_analysis']:
        raise ValueError(f"Layer {layer_idx} not found in the results.")

    # Get the MoE analysis for the specified layer
    moe_analysis = results['moe_analysis'][layer_idx]

    # Get the shared expert predictions for the specified token position
    shared_predictions = moe_analysis['shared_expert_predictions'][token_pos]
    
    # Return top 5 predictions
    return shared_predictions[:5]

def get_expert_preds(results: dict, layer_idx: int, token_pos: int) -> list:
    # Ensure the layer exists in the results
    if layer_idx not in results['moe_analysis']:
        raise ValueError(f"Layer {layer_idx} not found in the MoE analysis results.")
        
    # Get the MoE analysis for the specified layer
    expert_preds = []
    moe_analysis = results['moe_analysis'][layer_idx]
    expert_predictions = moe_analysis['expert_predictions_by_position']
    expert_toks = expert_predictions[token_pos]

    for expert_idx, preds in expert_toks.items():
        print(f'expert {expert_idx} : {preds}')
        expert_preds.append((int(expert_idx), preds))

    expert_preds = sorted(expert_preds, key=lambda x: x[0])
        
    return expert_preds

In [7]:
def get_highest_pred_combined(results: dict, model, tokenizer, post_attn_ln_inputs, res_stream: bool, layer_idx: int, token_pos: int, k: int = 6) -> list:
    """gets predictions from combining top k experts weighted by their router weights, excluding shared experts"""
    # Ensure layer exists in results
    if layer_idx not in results['moe_analysis']:
        raise ValueError(f"Layer {layer_idx} not found in the MoE analysis results.")
        
    # Get MoE analysis for the layer
    moe_analysis = results['moe_analysis'][layer_idx]
    
    # Get expert weights for this token
    expert_weights = {}
    selected_experts = moe_analysis['selected_experts'][token_pos]
    expert_weights_list = moe_analysis['expert_weights'][token_pos]
    
    if isinstance(selected_experts, int):
        # Handle case where only one expert is selected
        expert_weights[selected_experts] = 1.0
    else:
        # Handle case where multiple experts are selected with weights
        for expert_idx, weight in zip(selected_experts, expert_weights_list):
            # Skip shared experts
            if expert_idx < 0:  # Shared experts typically have negative indices
                continue
            expert_weights[expert_idx] = weight
                
    # Get top k experts by weight
    top_k_experts = sorted(expert_weights.items(), key=lambda x: x[1], reverse=True)[:k]
    print(f'Using top {k} experts: {[expert[0] for expert in top_k_experts]}')
    
    # Initialize combined hidden state
    combined_hidden_state = None
    
    # Combine hidden states from top k experts, weighted by their router weights
    for expert_idx, weight in top_k_experts:
        expert_hidden_state = moe_analysis['expert_hidden_states_by_position'][token_pos][expert_idx]['hidden_state']
        
        # Convert expert hidden state to tensor if it's a list
        if isinstance(expert_hidden_state, list):
            expert_hidden_state = torch.tensor(expert_hidden_state, dtype=torch.float16)
            
        # Weight the expert's hidden state by its router weight
        weighted_hidden_state = expert_hidden_state * weight
        
        if combined_hidden_state is None:
            combined_hidden_state = weighted_hidden_state
        else:
            combined_hidden_state += weighted_hidden_state
    
    # to include residual stream in the combined hidden state
    if res_stream == True:
        # Get residual stream
        residual = post_attn_ln_inputs[layer_idx][-1][0][0][token_pos]
        
        # Add residual to combined expert hidden states
        combined = combined_hidden_state + residual
    else:
        combined = combined_hidden_state
    
    # Get logits
    logits = model.lm_head(combined.unsqueeze(0))
    
    # Get top 5 predictions
    topk = torch.topk(logits[0], k=5)
    scores = topk.values.tolist()
    tokens = topk.indices.tolist()
    
    # Convert to token strings
    predictions = []
    for score, token in zip(scores, tokens):
        token_str = tokenizer.decode(token)
        predictions.append((token_str, score))
        
    return predictions

In [8]:
def get_top_expert_with_residual(results, model, tokenizer, post_attn_ln_inputs, layer_idx, token_pos, expert_idx):
    """Gets predictions for a specific expert after adding residual"""
    if layer_idx not in results['moe_analysis']:
        raise ValueError(f"Layer {layer_idx} not found in the MoE analysis results.")
        
    # Get MoE analysis for the layer
    moe_analysis = results['moe_analysis'][layer_idx]
    
    # Get expert hidden state
    if token_pos not in moe_analysis['expert_hidden_states_by_position']:
        return None
    if expert_idx not in moe_analysis['expert_hidden_states_by_position'][token_pos]:
        return None
        
    expert_hidden_state = moe_analysis['expert_hidden_states_by_position'][token_pos][expert_idx]['hidden_state']
    
    # Get residual stream
    residual = post_attn_ln_inputs[layer_idx][-1][0][0][token_pos]
    
    # Convert expert hidden state to tensor if it's a list
    if isinstance(expert_hidden_state, list):
        expert_hidden_state = torch.tensor(expert_hidden_state, dtype=torch.float16)
    
    # Add residual to expert hidden state
    combined = residual + expert_hidden_state
    
    # Get logits
    logits = model.lm_head(combined.unsqueeze(0))
    
    # Get top 5 predictions
    topk = torch.topk(logits[0], k=5)
    scores = topk.values.tolist()
    tokens = topk.indices.tolist()
    
    # Convert to token strings
    predictions = []
    for score, token in zip(scores, tokens):
        token_str = tokenizer.decode(token)
        predictions.append((token_str, score))
        
    return predictions

def get_top_experts_order(results, layer_idx, token_pos):
    """Gets the order of top experts by their weights"""
    if layer_idx not in results['moe_analysis']:
        return []
        
    moe_data = results['moe_analysis'][layer_idx]
    expert_weights = {}
    
    if token_pos >= len(moe_data['selected_experts']):
        return []
        
    experts = moe_data['selected_experts'][token_pos]
    weights = moe_data['expert_weights'][token_pos]
    
    for expert_idx, weight in zip(experts, weights):
        expert_weights[expert_idx] = weight
            
    # Sort experts by weight
    sorted_experts = sorted(expert_weights.items(), key=lambda x: x[1], reverse=True)
    return [expert_idx for expert_idx, _ in sorted_experts]

In [9]:
def create_logit_lens_viz(analysis_results, tokenizer, post_attn_ln_inputs, token_pos=0, color='sunset', layers_to_plot=[]):
    """Creates a modern heatmap visualization of predictions across model layers."""
    # Initialize data structures
    n_layers = max(analysis_results['layer_predictions'].keys()) + 1
    data = []
    all_values = []
    
    # Process each layer
    for layer in range(n_layers):
        if layers_to_plot and layer not in layers_to_plot:
            continue
            
        row = {
            'layer': layer,
            'l_out': None,
            'routed_combined': None,
            'top_exp_1': None,
            'top_exp_2': None,
            'top_exp_3': None,
            'top_exp_4': None,
            'top_exp_5': None,
            'top_exp_6': None,
            'predictions': {
                'l_out': [],
                'routed_combined': [],
                'top_exp_1': [],
                'top_exp_2': [],
                'top_exp_3': [],
                'top_exp_4': [],
                'top_exp_5': [],
                'top_exp_6': [],
            },
            'expert_ids': {},
            'expert_weights': {},
            'top_tokens': {
                'l_out': '',
                'routed_combined': '',
                'top_exp_1': '',
                'top_exp_2': '',
                'top_exp_3': '',
                'top_exp_4': '',
                'top_exp_5': '',
                'top_exp_6': '',
            }
        }
        
        # Layer predictions
        layer_preds = get_layer_predictions_for_token(analysis_results, layer, token_pos)
        if layer_preds:
            row['l_out'] = layer_preds[0][1]
            row['predictions']['l_out'] = layer_preds
            row['top_tokens']['l_out'] = layer_preds[0][0]
            all_values.append(layer_preds[0][1])
        
        # Expert predictions
        if layer in analysis_results['moe_analysis']:
            # Get combined routed experts prediction
            routed_preds = get_highest_pred_combined(analysis_results, model, 
                                                    tokenizer,
                                                    post_attn_ln_inputs,
                                                    res_stream=True, 
                                                    layer_idx=layer, 
                                                    token_pos=token_pos, 
                                                    k=6)
            if routed_preds:
                row['routed_combined'] = routed_preds[0][1]
                row['predictions']['routed_combined'] = routed_preds
                row['top_tokens']['routed_combined'] = routed_preds[0][0]
                all_values.append(routed_preds[0][1])
            
            # Get expert weights
            expert_weights = analysis_results['moe_analysis'][layer]['expert_weights'][token_pos]
            expert_indices = analysis_results['moe_analysis'][layer]['selected_experts'][token_pos]
            
            # Sort experts by weights
            expert_pairs = list(zip(expert_indices, expert_weights))
            sorted_experts = sorted(expert_pairs, key=lambda x: x[1], reverse=True)
            
            # Get individual expert predictions
            for i, (expert_idx, weight) in enumerate(sorted_experts[:6], 1):
                col_key = f'top_exp_{i}'
                expert_preds = get_top_expert_with_residual(analysis_results,
                                                            model, 
                                                            tokenizer,
                                                            post_attn_ln_inputs,
                                                            layer,
                                                            token_pos, 
                                                            expert_idx)
                if expert_preds:
                    row[col_key] = expert_preds[0][1]
                    row['predictions'][col_key] = expert_preds
                    row['top_tokens'][col_key] = expert_preds[0][0]
                    row['expert_ids'][col_key] = expert_idx
                    row['expert_weights'][col_key] = weight
                    all_values.append(expert_preds[0][1])

        data.append(row)
    
    valid_values = [v for v in all_values if v is not None]
    vmin = min(valid_values) if valid_values else 0
    vmax = max(valid_values) if valid_values else 1
    
    fig = go.Figure()
    
    columns = ['l_out', 'routed_combined'] + [f'top_exp_{i}' for i in range(1, 7)]
    x_positions = list(range(len(columns)))
    
    # Use separate colorscales for logits and weights
    logit_colorscale = color
    weight_colorscale = [[0, 'rgba(200,200,200,0)'], [1, 'rgba(120,0,0,1)']]  # Gray to darker red

    y_positions = list(range(len(data)))
    
    # Create base heatmap matrix
    z_matrix = []
    for row in data:
        z_row = []
        for col in columns:
            if row['layer'] == 0 and col != 'l_out':
                z_row.append(float('nan'))
            else:
                z_row.append(row[col] if row[col] is not None else float('nan'))
        z_matrix.append(z_row)
        
    # Add main heatmap
    fig.add_trace(go.Heatmap(
        z=z_matrix,
        x=x_positions,
        y=y_positions,
        colorscale=logit_colorscale,
        showscale=True,
        zmin=vmin,
        zmax=vmax,
        colorbar=dict(
            title='logit value',
            titleside='right',
            y=0.5,
            thickness=20,
            len=0.5,
            tickfont=dict(family='Inter', size=12),
            titlefont=dict(family='Inter', size=14)
        ),
        hoverongaps=False
    ))
    
    # Add overlays
    for i, row in enumerate(data):
        for j, col in enumerate(columns):
            if row['layer'] == 0 and col != 'l_out':
                continue
                
            if row[col] is not None:
                normalized_value = (row[col] - vmin) / (vmax - vmin) if vmax != vmin else 0
                text_color = 'white' if normalized_value > 0.5 else 'black'
                
                # Handle expert-specific elements
                if col.startswith('top_exp_'):
                    expert_id = str(row['expert_ids'].get(col, ''))
                    expert_weight = row['expert_weights'].get(col, 0)
                    
                    # Expert ID without border
                    fig.add_trace(go.Scatter(
                        x=[j - 0.4],
                        y=[i + 0.35],
                        mode='text',
                        text=[expert_id],
                        textposition='middle center',
                        textfont=dict(
                            family='Inter', 
                            color=text_color, 
                            size=12
                        ),
                        hoverinfo='skip',
                        showlegend=False
                    ))
                    
                    # Weight value with enhanced visibility
                    weight_text = f"{expert_weight:.3f}"
                    fig.add_trace(go.Scatter(
                        x=[j + 0.35],
                        y=[i - 0.35],
                        mode='text',
                        text=[weight_text],
                        textposition='middle center',
                        textfont=dict(
                            family='Inter', 
                            color=text_color, 
                            size=12
                        ),
                        hoverinfo='skip',
                        showlegend=False
                    ))
                    
                    # Weight indicator with separate color scale - now full width
                    fig.add_shape(
                        type="rect",
                        x0=j - 0.5,
                        y0=i - 0.45,
                        x1=j + 0.5,
                        y1=i - 0.25,
                        fillcolor=f'rgba(120,0,0,{expert_weight})',
                        line=dict(width=0),  # Remove border
                        layer='above'
                    )
                
                # Token text
                token_text = row['top_tokens'][col]
                fig.add_trace(go.Scatter(
                    x=[j],
                    y=[i],
                    mode='text',
                    text=[token_text],
                    textposition='middle center',
                    textfont=dict(
                        family='Inter', 
                        color=text_color, 
                        size=14
                    ),
                    hoverinfo='skip',
                    showlegend=False
                ))
    
    # Calculate dimensions
    n_rows = len(data)
    base_width = 1400
    min_height_per_row = 80
    plot_height = max(min_height_per_row * n_rows, 600)
    
    # Update layout
    fig.update_layout(
        xaxis=dict(
            ticktext=['layer output', 'routed exp + res'] + [f'top exp {i} + res' for i in range(1, 7)],
            tickvals=x_positions,
            tickangle=-45,
            showgrid=False,
            zeroline=False,
            title='',
            color='black',
            range=[-0.5, len(columns) - 0.5],
            constrain='domain',
            tickfont=dict(family='Inter', size=14),
            side='bottom',
            ticklen=10,
            tickmode='array',
            ticks='outside',
            showline=False,
            tickprefix='  '
        ),
        yaxis=dict(
            autorange='reversed',
            showgrid=False,
            zeroline=False,
            title=dict(
                text='Layer',
                font=dict(family='Inter', size=16)
            ),
            color='black',
            range=[-0.5, len(data) - 0.5],
            ticktext=[str(row['layer']) for row in data],
            tickvals=y_positions,
            tickfont=dict(family='Inter', size=14),
            constrain='domain',
            side='left',
            ticklen=10,
            tickmode='array',
            ticks='outside',
            showline=False,
            ticksuffix='  '
        ),
        plot_bgcolor='white',
        paper_bgcolor='white',
        width=base_width,
        height=plot_height,
        margin=dict(l=80, r=120, t=50, b=100),
        hovermode='closest'
    )
    
    # Add cell borders with enhanced visibility
    for i in range(len(data)):
        for j in range(len(columns)):
            fig.add_shape(
                type="rect",
                x0=j-0.5,
                y0=i-0.5,
                x1=j+0.5,
                y1=i+0.5,
                line=dict(color='rgba(0,0,0,0.2)', width=1),
                fillcolor="rgba(0,0,0,0)",
                layer="above"
            )
    
    # Add hover text
    for i, row in enumerate(data):
        for j, col in enumerate(columns):
            if row['layer'] == 0 and col != 'l_out':
                continue
                
            if row[col] is not None:
                preds = row['predictions'][col]
                if preds:
                    pred_text = "<br>".join([f"{token}: {score:.3f}" for token, score in preds[:5]])
                    if col.startswith('top_exp_'):
                        real_id = row['expert_ids'].get(col)
                        weight = row['expert_weights'].get(col, 0)
                        hover_text = f"Layer {row['layer']} expert {real_id} + residual (logit: {row[col]:.3f}, weight: {weight:.3f}):<br>{pred_text}"
                    else:
                        hover_text = f"Layer {row['layer']} {col} (logit: {row[col]:.3f}):<br>{pred_text}"
                    
                    fig.add_trace(go.Scatter(
                        x=[j],
                        y=[i],
                        mode='markers',
                        marker=dict(opacity=0),
                        hovertext=hover_text,
                        hoverinfo='text',
                        showlegend=False
                    ))
    
    return fig

---

In [28]:
prompt = '''the quick brown fox jumps '''
n = len(tokenizer.encode(prompt))

analysis = analyze_deepseek_moe(
    model, 
    tokenizer,
    input_text=prompt
)

In [11]:
post_attn_ln_inputs = get_post_attn_ln_inputs(model, tokenizer, text=prompt)

In [12]:
# Access layer predictions
for layer_idx, preds in analysis['layer_predictions'].items():
    print(f"Layer {layer_idx} predictions:", preds)

# Access MoE analysis
for layer_idx, moe_data in analysis['moe_analysis'].items():
    print(f"\nMoE Layer {layer_idx}:")
    print("Selected experts:", moe_data['selected_experts'])
    print("Expert weights:", moe_data['expert_weights'])
    print("Combined output tokens:", moe_data['combined_output_tokens'])
    print("Expert predictions by position:", moe_data['expert_predictions_by_position'])
    print("Shared expert predictions:", moe_data['shared_expert_predictions'])

Layer 0 predictions: [[('开辟', 1.318359375), ('晗', 1.244140625), ('wk', 1.2431640625), ('始终坚持', 1.2177734375), ('CM', 1.2080078125)], [('华盛', 1.12109375), ('AGA', 1.0419921875), ('OD', 1.03515625), ('ODO', 1.02734375), ('扭转', 0.99365234375)], [('UMP', 1.0712890625), (' QT', 1.0634765625), ('‐', 1.0341796875), ('OTS', 0.99169921875), ('ie', 0.9560546875)], [('ish', 1.392578125), ('�乐', 1.2607421875), ('ies', 1.15625), ('="../_', 1.099609375), ('wk', 1.0439453125)], [('">:', 1.17578125), ('anat', 1.1728515625), ('ed', 1.1455078125), ('soever', 1.083984375), ('es', 1.0751953125)], [('AO', 1.185546875), ('QC', 1.1337890625), ('QL', 1.0537109375), ('QA', 1.0361328125), ('/', 1.013671875)], [('开辟', 1.009765625), ('WN', 0.9443359375), ('晖', 0.9296875), ('颓', 0.921875), ('固执', 0.9208984375)]]
Layer 1 predictions: [[(' препратки', 91.0), ('不成', 88.625), ('扫', 88.4375), ('�乐', 88.375), ('idades', 88.0)], [('莫名其', 1.486328125), (' modernista', 1.3603515625), ('端正', 1.33984375), ('iso', 1.315429687

In [26]:
n = len(tokenizer.encode(prompt))
print(f'n : {n}')
layer_idx = 16
token_pos = n-1
k = 1 # which max expert to use
get_highest_pred_combined(analysis, model, tokenizer, post_attn_ln_inputs, res_stream = False, layer_idx=layer_idx, token_pos=token_pos, k=k)

n : 7
Using top 1 experts: [19]


[('9', 0.162109375),
 ('6', 0.1458740234375),
 ('5', 0.1329345703125),
 ('ilon', 0.12384033203125),
 ('坐', 0.1212158203125)]

In [29]:
layer_idx = 25
token_pos = n-1 # last token
layer_preds = get_layer_predictions_for_token(analysis, layer_idx=layer_idx, token_pos=token_pos)
shared_preds = get_shared_expert_predictions_for_token(analysis, layer_idx=layer_idx, token_pos=token_pos)
expert_preds = get_expert_preds(analysis, layer_idx=layer_idx, token_pos=token_pos)

print(f'layer_preds : {layer_preds}')
print(f'shared_preds : {shared_preds}')
print(f'expert_preds : {expert_preds}')

expert 1 : [('י', 14.734375), ('י�', 12.8828125), ('נ', 11.921875), ('س', 11.8984375), ('क', 11.609375)]
expert 2 : [('ime', 8.8359375), (' трудно', 8.734375), ('Английски', 8.5078125), (' ~[', 8.4609375), ('ameter', 8.4609375)]
expert 6 : [('ivil', 13.7109375), ('riend', 11.28125), ('icer', 11.234375), ('ript', 11.15625), ('ilon', 11.125)]
expert 40 : [('ub', 13.5078125), ('hot', 13.4453125), ('uites', 13.3046875), ('how', 12.4453125), ('uite', 12.140625)]
expert 56 : [(' vpon', 10.328125), ('\to', 9.7578125), ('ck', 9.3515625), ('Sentence', 8.8203125), ('不在', 8.6953125)]
expert 60 : [(' sole', 8.9921875), (' side', 8.9453125), ('ve', 8.6328125), (' care', 8.5546875), ('meas', 8.2421875)]
layer_preds : [('udisks', 20.6875), ('urant', 19.921875), ('icis', 19.6875), ('ille', 19.265625), ('rown', 19.171875)]
shared_preds : [(']{.', 4.921875), ('gredients', 4.5859375), ('ermita', 4.56640625), ('udisks', 4.51171875), ('^*,', 4.5078125)]
expert_preds : [(1, [('י', 14.734375), ('י�', 12.8828

In [16]:

# l = [0,2,6,8,13,18,21,22,25,27]
l = [] # empty list = all layers
x = len(tokenizer.encode(prompt))
print(x)
fig = create_logit_lens_viz(analysis, 
                            tokenizer,
                            post_attn_ln_inputs,
                            token_pos=x-1, 
                            color='blues', 
                            layers_to_plot=l)

# save the plot
i = 1
while os.path.exists(f'logit_lens_viz_{i}.html'):
    i += 1
fig.write_html(f'logit_lens_viz_{i}.html')
fig.write_image(f'logit_lens_viz_{i}.png')
fig.show()

7
Using top 6 experts: [40, 14, 46, 47, 28, 6]
Using top 6 experts: [15, 47, 61, 20, 2, 12]
Using top 6 experts: [25, 21, 60, 35, 31, 50]
Using top 6 experts: [52, 43, 49, 2, 31, 19]
Using top 6 experts: [37, 49, 40, 53, 58, 62]
Using top 6 experts: [60, 56, 55, 20, 24, 21]
Using top 6 experts: [12, 9, 45, 38, 61, 23]
Using top 6 experts: [45, 52, 62, 4, 32, 5]
Using top 6 experts: [0, 59, 41, 44, 9, 63]
Using top 6 experts: [16, 0, 1, 2, 40, 30]
Using top 6 experts: [45, 30, 54, 44, 3, 2]
Using top 6 experts: [48, 1, 33, 61, 54, 35]
Using top 6 experts: [18, 38, 55, 9, 23, 33]
Using top 6 experts: [44, 4, 10, 36, 2, 45]
Using top 6 experts: [27, 28, 18, 59, 2, 46]
Using top 6 experts: [19, 16, 40, 23, 36, 10]
Using top 6 experts: [57, 17, 42, 62, 38, 20]
Using top 6 experts: [49, 62, 7, 57, 43, 14]
Using top 6 experts: [10, 13, 7, 32, 11, 9]
Using top 6 experts: [55, 57, 25, 16, 50, 31]
Using top 6 experts: [42, 17, 45, 47, 31, 37]
Using top 6 experts: [34, 33, 56, 18, 7, 42]
Using to

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
